In [1]:
import re
import json
import requests
import csv
import datetime

# Replace the URL with the desired Shopee product URL
url = 'https://shopee.com.my/JISULIFE-3-in-1-Portable-Fan-9000-mAh-Battery-Rechargeable-With-Air-Duster-Function-Mini-Fan-With-1-100-Speeds-LED-Display-Outdoor-Camping-i.310060557.23154647628'

# Extract shop_id and item_id from the URL
r = re.search(r'i\.(\d+)\.(\d+)', url)
if r:
    shop_id, item_id = r.group(1), r.group(2)
else:
    print("Invalid URL")
    exit()

# Initialize offset and ratings_data list
offset = 0
ratings_data = []

# Loop to fetch ratings data, limited to 150 records
while offset < 150:
    # Construct ratings URL based on the provided documentation with a limit of 20 records per request
    ratings_url = f'https://shopee.sg/api/v2/item/get_ratings?shopid={shop_id}&itemid={item_id}&limit=20&offset={offset}&type=0&filter=0'

    # Fetch data from the API endpoint
    data = requests.get(ratings_url).json()

    # Extract relevant information from the API response
    for rating in data['data']['ratings']:
        author_username = rating['author_username']
        comment = rating['comment'].replace('\n', ' ')  # Replace newline characters with a space
        timestamp = rating['ctime']  # Extracting date information from the API response

        # Convert Unix timestamp to datetime object
        review_date = datetime.datetime.utcfromtimestamp(timestamp)
        # Format the datetime object as a string with the desired format '1-Jan-2024'
        formatted_review_date = review_date.strftime('%Y-%b-%d %H:%M:%S')

        ratings_data.append([author_username, formatted_review_date, comment])  # Append formatted review date

    # Break the loop if all desired records have been fetched
    if len(data['data']['ratings']) < 20:
        break

    offset += 20

# Write data to CSV file
csv_file_name = 'Shopee User Ratings (The Fan).csv'
with open(csv_file_name, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Name', 'Date', 'Content'])  # Write header with modified column names
    writer.writerows(ratings_data)

print(f"Data has been saved to {csv_file_name}.")


Data has been saved to Shopee User Ratings (The Fan).csv.
